A (maximal) clique in a graph $G$ is a complete subgraph of largest order in $G$. Notice that $\chi(G)$ is at least as large as the order of a clique denoted $\omega(G)$.

A greedy-type algorithm for finding a complete subgraph in G would start with a subgraph of order one (a vertex) and repeatedly try to find a vertex joined to all vertices of the subgraph selected so far, until no further such vertex could be found.

The greedy algorithm for finding a clique is myopic. It makes locally optimal choices without any foresight, which can lead it down a path that terminates prematurely.  

We can argue that it is unlikely to find a clique of order $14$ in a graph from $\mathcal{G}(2000, 0.5)$ by looking at the probability of being able to extend a clique at each step. Let's assume the algorithm has successfully found a clique of order $k$. To extend this to a clique of order $k+1$, it must find a vertex, from the remaining $n-k$ vertices, that is connected to all $k$ vertices already in the clique.

The probability that a specific vertex is connected to all $k$ vertices of the existing clique is $0.5^k$ by independence. The expected number of vertices that can extend the clique of size $k$ is given by the formula
\begin{equation}
    E = (2000 - k)(0.5)^k.
\end{equation}
If the algorithm has a clique of size 13, then the expected number of suitable vertices is $(2000 - 13)(0.5)^{13} = 1987 / 8192 \approx 0.24$. Also, note that once the algorithm has found a clique of size $11$, the expected number of vertices that can extend it drops below $1$.

Therefore, for the greedy algorithm to reach a clique of size 14, it would need to succeed against diminishing odds multiple times in a row. For a typical run starting from a random vertex, the algorithm will almost certainly terminate with a clique of size around $11$ or $12$, making it extremely unlikely to ever find one of order $14$.

While the greedy algorithm performs poorly, the actual largest clique in a $\mathcal{G}(2000, 0.5)$ graph is likely to be significantly larger. We can estimate this size by finding the value of $k$ for which we expect to find approximately one clique of that size in the entire graph.

The expected number of k-cliques is given by
\begin{equation}
    E[X_k] = C(n, k) p^{(k(k-1)/2)},
\end{equation}
where $C(n, k)$ is the number of ways to choose $k$ vertices from $n$. The size of the largest clique will be around the value of $k$ when $E[X_k] ≈ 1$. A well-known result in random graph theory approximates this value as $k \approx 2\log_{1/p}(n) \approx 21.94$.
This calculation suggests that the largest clique in a typical graph from $\mathcal{G}(2000, 0.5)$ is very likely to be of size $21$ or $22$.

In [1]:
import networkx as nx
import random

def generate_Gnp(n, p):
    '''
    Generates a random graph G(n, p).
    '''
    G = nx.Graph()
    G.add_nodes_from(range(n))
    for i in range(n):
        for j in range(i + 1, n):
            if random.random() < p:
                G.add_edge(i, j)
    return G

def generate_Gknp(k, n, p):
    '''
    Generates a random graph G_k(n, p).
    '''
    G = nx.Graph()
    G.add_nodes_from(range(n))
    for i in range(n):
        for j in range(i + 1, n):
            if (j - i) % k != 0:
                if random.random() < p:
                    G.add_edge(i, j)
    return G

In [2]:
def greedy_colouring(graph, order):
    '''
    Applies the greedy colouring algorithm and returns the number of colours.
    This gives an upper bound for the chromatic number.
    '''
    colours = {}
    for vertex in order:
        neighbor_colours = {colours.get(neighbor) for neighbor in graph.neighbors(vertex)}
        colour = 1
        while colour in neighbor_colours:
            colour += 1
        colours[vertex] = colour
    return max(colours.values()) if colours else 0

def order_by_increasing_degree(graph):
    return sorted(graph.nodes(), key=lambda v: graph.degree(v))

def order_by_decreasing_degree(graph):
    return sorted(graph.nodes(), key=lambda v: graph.degree(v), reverse=True)

def order_smallest_last(graph):
    g_copy = graph.copy()
    ordering = []
    while g_copy.number_of_nodes() > 0:
        min_degree_vertex = min(g_copy.nodes(), key=lambda v: g_copy.degree(v))
        ordering.append(min_degree_vertex)
        g_copy.remove_node(min_degree_vertex)
    return ordering[::-1]

def order_at_random(graph):
    nodes = list(graph.nodes())
    random.shuffle(nodes)
    return nodes

def find_clique(graph):
    '''
    Finds a large clique using a repeated greedy search starting from each vertex.
    The size of this clique is a lower bound for the chromatic number.
    '''
    nodes = list(graph.nodes())
    random.shuffle(nodes)
    best_clique_found = []
    for start_node in nodes:
        current_clique = [start_node]
        candidates = list(graph.neighbors(start_node))
        random.shuffle(candidates)
        for candidate_node in candidates:
            is_fully_connected = all(graph.has_edge(candidate_node, member) for member in current_clique)
            if is_fully_connected:
                current_clique.append(candidate_node)
        if len(current_clique) > len(best_clique_found):
            best_clique_found = current_clique

    return best_clique_found

In [3]:
def run_comparison_experiment(graph_generator, params, description):
    '''
    Generates a graph, finds lower and upper bounds for χ(G), and prints the comparison.
    '''
    print("=" * 45)
    print(f"Bounds for Chromatic Number in {description}")
    print("=" * 45)

    for i in range(3):
        print(f"Graph Sample {i+1}")
        G = graph_generator(*params)
        clique = find_clique(G)
        lower_bound = len(clique)
        ordering_strategies = {
            "Increasing Degree": order_by_increasing_degree,
            "Decreasing Degree": order_by_decreasing_degree,
            "Smallest Last": order_smallest_last,
            "Random": order_at_random
        }
        upper_bounds = []
        for name, func in ordering_strategies.items():
            order = func(G)
            num_colours = greedy_colouring(G, order)
            upper_bounds.append(num_colours)

        best_upper_bound = min(upper_bounds)
        print(f"  {lower_bound} ≤ χ(G) ≤ {best_upper_bound}")

In [4]:
run_comparison_experiment(generate_Gnp, (70, 0.5), "G(70, 0.5)")
run_comparison_experiment(generate_Gknp, (3, 70, 0.75), "G_3(70, 0.75)")

Bounds for Chromatic Number in G(70, 0.5)
Graph Sample 1
  8 ≤ χ(G) ≤ 15
Graph Sample 2
  7 ≤ χ(G) ≤ 15
Graph Sample 3
  8 ≤ χ(G) ≤ 15
Bounds for Chromatic Number in G_3(70, 0.75)
Graph Sample 1
  3 ≤ χ(G) ≤ 3
Graph Sample 2
  3 ≤ χ(G) ≤ 3
Graph Sample 3
  3 ≤ χ(G) ≤ 3


The procedure successfully establishes a range for the true chromatic number $\chi(G)$. For the first sample of $\mathcal{G}(70, 0.5)$, we can confidently say that $\chi(G)$ is between $7$ and $16$.

For the denser $\mathcal{G}_3(70, 0.75)$ graphs, the bounds are often much tighter. In the example output, the range $3 \leq \chi(G) \leq 3$ is very narrow, giving us a very good estimate of the true chromatic number.

This demonstrates the classic relationship in graph theory: $\omega(G) ≤ \chi(G) ≤ \Delta(G) + 1$, where $\omega(G)$ is the clique number and $\Delta(G)$ is the maximum degree. Our greedy colouring upper bounds are often much better than the general $\Delta(G) + 1$ upper bound, and the clique search provides a non-trivial lower bound.